In [31]:
import numpy as np
from numpy.linalg import lstsq, inv
from scipy.linalg import sqrtm
from scipy.sparse import diags, csr_matrix, kron
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from functools import reduce

init_notebook_mode(connected=True) 
np.random.seed(42)

# helper functions
def addVertLinePlotly(fig, x0=0, y0=0, y1=1):
    """ plots a vertical line to the given figure at position x"""
    fig.add_shape(dict(type="line", x0=x0, x1=x0, y0=y0, y1=1.2*y1, 
                       line=dict(color="LightSeaGreen", width=1)))
    return


def secondOrderDiff(y):
    """ calculate the 2nd order difference for a given series y """
    a = [y[i] - 2*y[i+1] + y[i+2] for i in range(len(y)-2)]
    return a 

In [8]:
import numpy as np

class TestFunctions:
    """
    Collection of test functions.
    
    ...
    
    Attributes
    ----------
    x1_min : float
             Minimum value of axis 1. 
    x1_max : float
             Maximum value of axis 1.
    x2_min : float
             Minimum value of axis 2.
    x2_max : float
             Maximum value of axis 2.
    n_samples : int
                Number of samples to evaluate
    noise_level : float
                  Specify the magnitude of noise influence.
    

    Methods
    ---------
    f1(x)
        tanh + exp + noise
    f2( )
        peak + noise (optional)
    f3(x)
        sin + linear part + noise
    f4(x1, x2)
        affensattel
    """
    def __init__(self, x1_min=0, x1_max=1, x2_min=0, x2_max=1, n_samples=1000, noise_level=0.25):
        self.x1_min = x1_min
        self.x1_max = x1_max
        self.x2_min = x2_min
        self.x2_max = x2_max
        self.n_samples = n_samples
        self.noise_level = noise_level
        
    def f1(self):
        x = np.linspace(self.x1_min, self.x1_max, self.n_samples)
        y = np.tanh(-(x-5)) + np.exp(-(x)**2) + np.random.randn(len(x))*self.noise_level
        return x, y
    
    def f2(self, a=1, b=1):
        x = np.linspace(self.x1_min, self.x1_max, self.n_samples)
        y = a / (1 + (b*x)**2) + np.random.randn(len(x))*self.noise_level
        return x, y
    
    def f3(self): 
        x = np.linspace(self.x1_min, self.x1_max, self.n_samples)
        y = 0.5*np.sin(x) + 3.5*np.linspace(0,1,len(x)) + np.random.randn(len(x))*self.noise_level
        return x, y
    
    def f4(self, grid=True):
        """Evaluate the 2-d function on a grid """
        x1 = np.linspace(self.x1_min, self.x1_max, self.n_samples)
        x2 = np.linspace(self.x2_min, self.x2_max, self.n_samples)
        y = x1**3 - 3*x1*x2**2 + np.random.randn(len(x1))*self.noise_level
        return x, y

        

In [5]:
# test functions
def f1(x): 
    return np.tanh(-(x-5)) + np.exp(-(x)**2) + np.random.randn(len(x))*0.1

def f2(xmin=0, xmax=1, n=1000, a=1, b=1, noiseType="gauss"): 
    """ either gaussian noices or decreasing noise"""
    x = np.linspace(xmin,xmax, n)
    noise = []
    for i in x:
        noise.append(np.random.normal(0, (xmax - i)/(xmax - xmin)*0.25))
    
    if noiseType is "gauss":
        return x, a / (1 + (b*x)**2) + np.random.randn(len(x))*0.25
    else:
        return x, a / (1 + (b*x)**2) + noise
    
def f3(x): 
    return 0.5*np.sin(x) + 3.5*np.linspace(0,1,len(x)) + np.random.randn(len(x))*0.2

def f4(x1, x2): 
    return x1**3 - 3*x1*x2**2 #+ np.random.randn(len(x1))*0.1


In [9]:
import numpy as np
tf = TestFunctions()
tf.f1()

(array([0.        , 0.001001  , 0.002002  , 0.003003  , 0.004004  ,
        0.00500501, 0.00600601, 0.00700701, 0.00800801, 0.00900901,
        0.01001001, 0.01101101, 0.01201201, 0.01301301, 0.01401401,
        0.01501502, 0.01601602, 0.01701702, 0.01801802, 0.01901902,
        0.02002002, 0.02102102, 0.02202202, 0.02302302, 0.02402402,
        0.02502503, 0.02602603, 0.02702703, 0.02802803, 0.02902903,
        0.03003003, 0.03103103, 0.03203203, 0.03303303, 0.03403403,
        0.03503504, 0.03603604, 0.03703704, 0.03803804, 0.03903904,
        0.04004004, 0.04104104, 0.04204204, 0.04304304, 0.04404404,
        0.04504505, 0.04604605, 0.04704705, 0.04804805, 0.04904905,
        0.05005005, 0.05105105, 0.05205205, 0.05305305, 0.05405405,
        0.05505506, 0.05605606, 0.05705706, 0.05805806, 0.05905906,
        0.06006006, 0.06106106, 0.06206206, 0.06306306, 0.06406406,
        0.06506507, 0.06606607, 0.06706707, 0.06806807, 0.06906907,
        0.07007007, 0.07107107, 0.07207207, 0.07

In [4]:
#######################################################################
# trusted
#######################################################################
# BSpline functionality    
#######################################################################
def bSpline(x, k, i, m=2):
    """ 
    evaluate i-th b-spline basis function of order m at the 
    values in x, given knot loactions in k
    """
    if m==-1:
        # return 1 if x is in {k[i], k[i+1]}, otherwise 0
        return (x >= k[i]) & (x < k[i+1]).astype(int)
    else:
        #print("m = ", m, "\t i = ", i)
        z0 = (x - k[i]) / (k[i+m+1] - k[i])
        z1 = (k[i+m+2] - x) / (k[i+m+2] - k[i+1])
        return z0*bSpline(x, k, i, m-1) + z1*bSpline(x, k, i+1, m-1)

# trusted
def modelMatrixBSpline(x, k=0, m=2):
    """ 
    set up model matrix for the B-spline basis
    one needs k + m + 1 knots for a spline basis of order m with k 
    parameters,
    k - number of parameters (== number of B-splines)
    """
    n = len(x) # n = number of data
    assert (k > 0), "number of B-splines k need to be specified and larger than 0"
    xmin, xmax = np.min(x), np.max(x)
    xk = np.quantile(a=x, q=np.linspace(0,1,k))
    dx = xk[m+3] - xk[m+2]
    xk = np.insert(xk, 0, np.arange(xmin-(m+1)*dx, xmin, dx))
    xk = np.append(xk, np.arange(xmax+dx, xmax+(m+2)*dx, dx))
    X = np.zeros(shape=(n, k))
    for i in range(k):
        X[:,i] = bSpline(x=x, k=xk, i=i+1, m=m)
    return X, xk

#trusted
def pSplinePenalty(k):
    """ 
    compute the P-spline second order difference penalty matrix of 
    dimension k according to p.150 Wood 2006 
    """
    assert (type(k) is int), "Type of input k is not int"
    a =np.eye(k, dtype=np.int)
    P = np.diff(a, axis=0)
    S = P.T @ P
    return S

def plot_ModelMatrixBSpline(x, k, m=2):
    X, xk = modelMatrixBSpline(x, k=k, m=m)

    fig = go.Figure()
    for i in range(X.shape[1]):
        fig.add_trace(go.Scatter(x=x, y=X[:,i], 
                                 name=f"BSpline {i+1}", mode="lines"))
    for i in xk:
        addVertLinePlotly(fig, x0=i)
    fig.show()
    return

In [5]:
#def B_Splines
k, m = 10, 2
x = np.linspace(-100, 10,1000)

plot_ModelMatrixBSpline(x, k)

In [12]:
# trusted
def iterativeFit(x, y, k, type_="mi", lam_p=0.1, lam_c = 1e3, extremumIdx=None):
    # get B-Spline basis
    X, xk = modelMatrixBSpline(x, k=k)
    beta = np.zeros(X.shape[1])
    
    yExt = x[np.argmax(y)] if type_ is "peak" else x[np.argmin(y)]
    extremumIdx = int(np.argwhere(xk > yExt)[0]- 1)
    # print(f"Shape of model matrix: {X.shape}")
    # get P-Spline penalty matrix
    P_matrix = D1_differenceMatrix(k)
    # print(f"Shape of smooth-penalty matrix: {P_matrix.shape}")
    # get monotonicity penalty and weight matrix
    D1 = D1_differenceMatrix(k=k)    
    D2 = D2_differenceMatrix(k=k)
    
    if type_ in ["mi", "md"]:
        D = D1
    else:
        D = D2
        
    beta_hat = inv(X.T@X + lam_p * P_matrix.T @ P_matrix) @ X.T @ y
    y_hat = X @ beta_hat
    error = mse(y, y_hat)

    for i in range(3):
        V = V_weightMatrix(k=k, type_=type_, beta=beta_hat, extremumIdx=extremumIdx)
        xdot = X.T @ X
        pdot = P_matrix.T @ P_matrix
        ddot = D.T @ V @ D
        beta_hat = inv(xdot + lam_p * pdot + lam_c * ddot) @ X.T @ y
        y_hat = X @ beta_hat
        error = mse(y, y_hat)
        #print("Nonzeros in V: ", np.count_nonzero(np.diag(V)))
        #print(f"MSE: {np.round(error, 5)}")
    
    return beta_hat, y_hat

In [13]:
#######################################################################
# trusted
#######################################################################
# PENALTY MATRICES
#######################################################################
def V_weightMatrix(k=0, type_="mi", beta=None, extremumIdx=None):
    """
    calcualte the weight matrix for the different types of penalty,
    e.g. md, mi, convex, concave
    """
    notGivenError = " are of wrong type or not given!"
    if type_ is "mi":
        d = (np.diff(beta) < 0).astype(int)
        d = np.append(d, 0)
        addZeros = np.zeros(k)
    elif type_ is "md":
        d = (np.diff(beta) > 0).astype(int)
        d = np.append(d, 0)
        addZeros = np.zeros(k)
    elif type_ is "convex":
        assert (beta is not None), "beta"+notGivenError
        D2 = D2_differenceMatrix(k=k)
        d = (D2 @ beta < 0).astype(int)
    elif type_ is "concave":
        assert (beta is not None), "beta"+notGivenError
        D2 = D2_differenceMatrix(k=k)
        d = (D2 @ beta > 0).astype(int)
    elif type_ is "peak":
        assert (extremumIdx is not None), "extremumIdx"+notGivenError
        P = Peak_Matrix(k=k, peakIdx=extremumIdx)
        d = (P @ beta < 0).astype(int)
    elif type_ is "valley":
        assert (extremumIdx is not None), "extremumIdx"+notGivenError
        V = Valley_Matrix(k=k, valleyIdx=extremumIdx)
        d = (V @ beta < 0).astype(int)
    else:
        print(f"Requested type {type_} not implemented so far!")
        d = np.ones(k)
    
    offset = 0
    V = diags(d, offset, dtype=np.int).toarray()
    #print("Shape of V-Matrix: {}".format(V.shape))
   
    return V

# trusted
def Peak_Matrix(k=0, peakIdx=0):
    """ 
    calculate the peak penalty matrix for monotonic increasing
    and monotonic decreasing peak
    """
    assert (type(k) is int), "Type of input k is not int"
    assert (type(peakIdx) is int), "Type of input peakIdx is not int"

    dia = -1*np.ones(k)
    dia[peakIdx] = 0
    #dia[peakIdx-1:peakIdx+1] = 0
    dia[peakIdx+1:] = 1
    offdia = -1 * dia
    offset = [0,1]
    Peak = diags(np.array([dia, offdia]), offset, dtype=np.int).toarray()
    Peak[-1:] = 0
    return Peak

# trusted
def Valley_Matrix(k=0, valleyIdx=0):
    """ 
    calculate the peak penalty matrix for monotonic increasing
    and monotonic decreasing peak
    """
    assert (type(k) is int), "Type of input k is not int"
    assert (type(valleyIdx) is int), "Type of input peakIdx is not int"

    dia = 1*np.ones(k)
    dia[valleyIdx] = 0
    #dia[peakIdx-1:peakIdx+1] = 0
    dia[valleyIdx+1:] = -1
    offdia = -1 * dia
    offset = [0,1]
    Valley = diags(np.array([dia, offdia]), offset, dtype=np.int).toarray()
    Valley[-1:] = 0
    return Valley

# trusted
def D1_differenceMatrix(k=0):
    """
    calculated the first order difference matrix  
    returns a matrix of size [k x k-1]
    """
    assert (type(k) is int), "Type of input k is not int"
    d = np.array([-1*np.ones(k), np.ones(k)])
    offset=[0,1]
    D1 = diags(d,offset, dtype=np.int).toarray()
    D1[-1:] = 0.
    #print("Shape of D1-Matrix: {}".format(D1.shape))
    return D1

# trusted
def D2_differenceMatrix(k=0):
    """
    calculated the second order difference matrix  
    returns a matrix of size [k x k-1]
    """
    assert (type(k) is int), "Type of input k is not int"
    d = np.array([np.ones(k), -2*np.ones(k), np.ones(k)])
    offset=[0,1,2]
    D2 = diags(d,offset, dtype=np.int).toarray()
    D2[-2:] = 0.
    #print("Shape of D2-Matrix: {}".format(D2.shape))
    return D2

In [7]:
D1 = D1_differenceMatrix(k=10)
beta = np.random.rand(10)
print("Beta: \n", beta)
D1 @ beta

Beta: 
 [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258]


array([ 5.76174188e-01, -2.18720365e-01, -1.33335458e-01, -4.42639844e-01,
       -2.41201062e-05, -9.79109082e-02,  8.08092534e-01, -2.65061134e-01,
        1.06957566e-01,  0.00000000e+00])

In [14]:
constrDict = {"smooth": 0, "mi":0, "md":0, "convex":0, "concave":0}
def additiveModel_Matrix(x1, x2, k, y=None):
    """ 
    calculate the model matrix for P-splines for more than one
    dimension
    """
    x1, x2 = np.sort(x1), np.sort(x2)
    X1, X2 = modelMatrixBSpline(x1, k=k), modelMatrixBSpline(x2, k=k)
    X = np.hstack((X1[0], X2[0]))
    SP1, SP2 = sqrtm(pSplinePenalty(k=k)), sqrtm(pSplinePenalty(k=k))
    SP = np.block([[SP1, np.zeros((SP1.shape[0], SP2.shape[1]))],
                   [np.zeros((SP2.shape[0], SP1.shape[1])), SP2]])
    X = np.vstack((X, SP))
    
    if type(y) is None:
        return X.astype(np.float)
    else:
        y = np.append(y, np.zeros(2*k))
        return X.astype(np.float), y

def penalizedModelMatrix(x, y, k=0, b0=None):
    assert (b0 is not None), "Give initial guess for parameter vector beta!"
    penalties = np.array(
        [pSplinePenalty(k=k), 
                 D1_differenceMatrix(k=k).T @ V_weightMatrix(k, "mi", b0, None) @ D1_differenceMatrix(k),
                 D1_differenceMatrix(k=k).T @ V_weightMatrix(k, "md", b0, None) @ D1_differenceMatrix(k),
                 D2_differenceMatrix(k=k).T @ V_weightMatrix(k, "convex", b0, None) @ D2_differenceMatrix(k),
                 D2_differenceMatrix(k=k).T @ V_weightMatrix(k, "concave", b0, None) @ D2_differenceMatrix(k),
        ])
    lam = np.array(list(constrDict.values())).reshape((len(constrDict.values()),-1))
    p = [i * lam[c] for c, i in enumerate(penalties)]
    p2 = list(map(sqrtm, p))
    p3 = [p2[i].astype(np.int) if m==0 else p2[i].astype(np.float) 
          for i,m in enumerate(lam)]

    X = modelMatrixBSpline(x, k)[0]
    prod = reduce((lambda x1,x2: np.vstack((x1,x2))), p3)
    X_aug = np.vstack((X, prod))
    y_aug = np.append(y, np.zeros(int(len(penalties)*k)))
    
    return X_aug, y_aug

def fit_additiveModel_2d(x1, x2, y, k=1, plot_=False):
    """
    fit an additive model for 2 dimensions using P-Splines
    """
    X, y_aug = additiveModel_Matrix(x1=x1, x2=x2, k=k, y=y)
    beta_hat = lstsq(a=X, b=y_aug)[0]
    error = mse(y_aug, X @ beta_hat)
    print(f"Mean Squared Error: {error}")
    if plot_:
        fig = make_subplots(rows=2, cols=2,
                            specs=[[{}, {}], 
                                   [{"colspan": 2, "type":"scene"}, None]],)
        fig.add_trace(go.Scatter(x=x1, y=y, name="data", 
                                 mode="markers"), row=1, col=1)
        fig.add_trace(go.Scatter(x=x1, y=X@beta_hat, mode="markers+lines",
                                 name="Fit"), row=1, col=1)
        
        fig.add_trace(go.Scatter(x=x2, y=y, name="data", 
                                 mode="markers"), row=1, col=2)
        fig.add_trace(go.Scatter(x=x2, y=X@beta_hat, mode="markers+lines",
                                 name="Fit"), row=1, col=2)
        
        fig.add_trace(go.Scatter3d(x=x1, y=x2, z=y, name="data",
                                   mode="markers"), row=2, col=1)
        fig.add_trace(go.Scatter3d(x=x1, y=x2, z=X@beta_hat, name="fit", 
                                   mode="markers"), row=2, col=1)
        fig.update_traces(marker=dict(size=2))
        fig.show()
    return

def fit_additiveModel_2d_with_penalties(x1, x2, y, k=1, plot_=False):
    
    X1_pre = modelMatrixBSpline(x1, k=k)[0]
    X2_pre = modelMatrixBSpline(x2, k=k)[0]
    print("Shape of modelmatrix X1: ", X1_pre.shape)
    print("Shape of modelmatrix X2: ", X2_pre.shape)
    print("Shape of y: ", y.shape)
    b1_pre = lstsq(X1_pre, y)[0]
    b2_pre = lstsq(X2_pre, y)[0]
    
    X1, y1 = penalizedModelMatrix(x1, y, k=k, b0=b1_pre)
    X2, y2 = penalizedModelMatrix(x2, y, k=k, b0=b2_pre)
    X = np.hstack((X1, X2))
    print("Shape of the final model matrix: ", X.shape)
    print("Shape y1: ", y1.shape)
    print("Shape y2: ", y2.shape)
    y = y1 if np.sum(y1 - y2) < 1e-8 else print("Error")
    beta = lstsq(a=X, b=y)[0]
    
    if plot_:
        fig = make_subplots(rows=2, cols=2,
                            specs=[[{}, {}], 
                                   [{"colspan": 2, "type":"scene"}, None]],)
        fig.add_trace(go.Scatter(x=x1, y=y, name="data", 
                                 mode="markers"), row=1, col=1)
        fig.add_trace(go.Scatter(x=x1, y=X@beta, mode="markers+lines",
                                 name="Fit"), row=1, col=1)

        fig.add_trace(go.Scatter(x=x2, y=y, name="data", 
                                 mode="markers"), row=1, col=2)
        fig.add_trace(go.Scatter(x=x2, y=X@beta, mode="markers+lines",
                                 name="Fit"), row=1, col=2)

        fig.add_trace(go.Scatter3d(x=x1, y=x2, z=y, name="data",
                                   mode="markers"), row=2, col=1)
        fig.add_trace(go.Scatter3d(x=x1, y=x2, z=X@beta, name="fit", 
                                   mode="markers"), row=2, col=1)
        fig.update_traces(marker=dict(size=2))
        fig.show()
    
    return beta

In [15]:
#x, y = f2(xmin=-3, xmax=3, noiseType="nope")
x = np.linspace(0,9,1000)
y = f3(x) + 0.5 * np.random.randn(len(x))

beta_hat, y_hat = iterativeFit(x, y, k=24, type_="mi", lam_p=1, lam_c=0)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=x, y=y, name="data", mode="markers"), row=1, col=1)
fig.add_trace(go.Scatter(x=x, y=y_hat, name="Fit"), row=1, col=1)
fig.add_trace(go.Scatter(x=x[:-1], y=np.diff(y_hat), name="Differenz in y"), row=2, col=1)
fig.update_yaxes(title_text="", type="log", row=2, col=1)

In [16]:
def fit_1d(x, y, k=1, smooth=True, plot_=False):
    """
    """
    if smooth:
        beta_init = np.random.random(k)
        Xa, ya = penalizedModelMatrix(x, y, k=k, b0=beta_init)
    else:
        Xa, xk = modelMatrixBSpline(x=x, k=k)
        ya = y
    #print("Shape Xa: ", Xa.shape)
    #print("Shape ya: ", ya.shape)
    beta_hat = lstsq(a=Xa, b=ya)[0]
    error = mse(ya, Xa @ beta_hat)
    print(f"Mean Squared Error: {error}")
    if plot_:
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x1, y=y, name="data", 
                                 mode="markers"))
        fig.add_trace(go.Scatter(x=x1, y=Xa@beta_hat, mode="lines",
                                 name="Fit"))
        fig.update_traces(marker=dict(size=2))
        fig.show()
    return beta_hat, Xa

In [58]:
ne = 10
k = 6
x = np.linspace(0,3,ne)
y = np.linspace(0,5,ne)
xg, yg = np.meshgrid(x, y)
X = modelMatrixBSpline(x, k)[0]
Y = modelMatrixBSpline(y, k)[0]

K = kron(X,Y).toarray()
print("X: ", xg.shape)
print("Y :", yg.shape)
print("K :", K.shape)
fig = go.Figure()
for i in range(K.shape[1]):
    fig.add_trace(go.Surface(x=xg, y=yg, z=K[:,i].reshape((ne, ne))))
fig.show()

X:  (10, 10)
Y : (10, 10)
K : (100, 36)


C:\Users\WeberJ\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:14: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



[-0.796  2.186  0.496  2.527  0.137  1.934]
(50, 6)


Mean Squared Error: 0.12823002850465778
[0.483 1.106 1.486 1.537 1.215 0.656]
[-0.796  2.186  0.496  2.527  0.137  1.934]


C:\Users\WeberJ\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:12: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



In [18]:
!pip install ndsplines

In [19]:
!pip install cython

In [54]:
# Test out ndsplines 0.1.1 
# https://pypi.org/project/ndsplines/

import ndsplines
import numpy as np
from scipy import interpolate
import numpy as np

NUM_X = 10
NUM_Y = 10
x = np.linspace(-3, 3, NUM_X)
y = np.linspace(-3, 3, NUM_Y)
meshx, meshy = np.meshgrid(x,y, indexing='ij')
input_coords = np.stack((meshx, meshy), axis=-1)
K = np.array([[1, -0.7,], [-0.7, 1.5]])
meshz = np.exp(-np.einsum(K, [1,2,], input_coords, [...,1], input_coords, [...,2])) + 0.1 * np.random.randn(NUM_X,NUM_Y)
#print("meshx: \n", meshx)
#print("meshy: \n", meshy)
#print("input_coords: \n", input_coords)
xt = [-1, 0, 1]
yt = [-1, 0, 1]
k = 3
xt = np.r_[(x[0],)*(k+1), xt, (x[-1],)*(k+1)]
yt = np.r_[(y[0],)*(k+1), yt, (y[-1],)*(k+1)]
ts = [xt, yt]

samplex = input_coords.reshape((-1,2))
sampley = meshz.reshape((-1))

spl = ndsplines.make_lsq_spline(samplex, sampley, ts, np.array([3,3]))

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=meshx.flatten(), 
                           y=meshy.flatten(), 
                           z=meshz.flatten(), mode="markers", name="data"))
fig.add_trace(go.Surface(x=meshx, y=meshy, z=spl(input_coords), name="fit"))
fig.update_traces(marker=dict(size=4), selector=dict(mode="markers"))
fig.show()